In [1]:
!pip install --upgrade openai

import openai
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [2]:
from google.colab import userdata
open_ai_api_key = userdata.get('OPENAI_API_KEY')

In [3]:
import os
os.environ["OPENAI_API_KEY"] = open_ai_api_key

In [4]:
openai.api_key = os.environ['OPENAI_API_KEY']

In [22]:
import json

In [18]:
# A mock function to fetch weather information
def get_weather(location, date):
    return {
        "location": location,
        "date": date,
        "forecast": "Sunny with a high of 25°C and a low of 15°C"
    }


In [19]:
# Define the function schema
functions = [
    {
        "name": "get_weather",
        "description": "Retrieve the weather forecast for a given location and date.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "The city or location to get the weather for."},
                "date": {"type": "string", "description": "The date for the weather forecast (YYYY-MM-DD)."}
            },
            "required": ["location", "date"]
        }
    }
]


In [20]:
# Prompt the model with a user query
messages = [
    {"role": "user", "content": "What's the weather like in New York City tomorrow?"}
]

# Call OpenAI with function calling
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
    function_call="auto",  # Allow the model to decide to call the function
    max_tokens=100
)

# Print the model's response
print("Response:", response.choices[0].message)

Response: ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"location":"New York City","date":"2023-02-20"}', name='get_weather'), tool_calls=None)


In [23]:
if hasattr(response.choices[0].message, "function_call"):
    function_name = response.choices[0].message.function_call.name
    arguments = response.choices[0].message.function_call.arguments
    parsed_arguments = json.loads(arguments)  # Ensure json is imported

    # Call the appropriate function
    if function_name == "get_weather":
        function_response = get_weather(**parsed_arguments)
        print("Function Response:", function_response)
else:
    print("No function call was made.")

Function Response: {'location': 'New York City', 'date': '2023-02-20', 'forecast': 'Sunny with a high of 25°C and a low of 15°C'}
